# 手撕 Flash Attention Pytorch实现

![](./flash.png)

## 1. 设定分块矩阵

In [3]:
import torch
from einops import rearrange

NEG_INF = -1e10  # -infinity
EPSILON = 1e-10

Q_LEN = 6
K_LEN = 6
Q_BLOCK_SIZE = 3 # 
KV_BLOCK_SIZE = 3
Tr = Q_LEN // Q_BLOCK_SIZE
Tc = K_LEN // KV_BLOCK_SIZE

Q = torch.randn(1, 1, Q_LEN, 4, requires_grad=True).to(device='cpu')
K = torch.randn(1, 1, K_LEN, 4, requires_grad=True).to(device='cpu')
V = torch.randn(1, 1, K_LEN, 4, requires_grad=True).to(device='cpu')
O = torch.zeros_like(Q, requires_grad=True)
l = torch.zeros(Q.shape[:-1])[..., None]
m = torch.ones(Q.shape[:-1])[..., None] * NEG_INF

Q_BLOCKS = torch.split(Q, Q_BLOCK_SIZE, dim=2)
K_BLOCKS = torch.split(K, KV_BLOCK_SIZE, dim=2)
V_BLOCKS = torch.split(V, KV_BLOCK_SIZE, dim=2)
O_BLOCKS = list(torch.split(O, Q_BLOCK_SIZE, dim=2))
l_BLOCKS = list(torch.split(l, Q_BLOCK_SIZE, dim=2))
m_BLOCKS = list(torch.split(m, Q_BLOCK_SIZE, dim=2))

## 2. 计算Flash Attention

In [5]:
# 先 KV 后 Q 
for j in range(Tc):
    Kj = K_BLOCKS[j]
    Vj = V_BLOCKS[j]
    for i in range(Tr):
        Qi = Q_BLOCKS[i]
        Oi = O_BLOCKS[i]
        li = l_BLOCKS[i]
        mi = m_BLOCKS[i]

        S_ij = torch.einsum('... i d, ... j d -> ... i j', Qi, Kj)
        m_block_ij, _ = torch.max(S_ij, dim=-1, keepdims=True)
        P_ij = torch.exp(S_ij - m_block_ij)
        l_block_ij = torch.sum(P_ij, dim=-1, keepdims=True) + EPSILON
        P_ij_Vj = torch.einsum('... i j, ... j d -> ... i d', P_ij, Vj)
        
        mi_new = torch.maximum(m_block_ij, mi)
        
        li_new = torch.exp(mi - mi_new) * li  \
               + torch.exp(m_block_ij - mi_new) * l_block_ij 

        O_BLOCKS[i] = (li / li_new) * torch.exp(mi - mi_new) * Oi \
                    +(torch.exp(m_block_ij - mi_new) / li_new) * P_ij_Vj
        print(f'-----------Attn : Q{i}xK{j}---------')
#         print(O_BLOCKS[i].shape)
        print(O_BLOCKS[0])
        print(O_BLOCKS[1])
        print('\n')
        
        l_BLOCKS[i] = li_new
        m_BLOCKS[i] = mi_new

O = torch.cat(O_BLOCKS, dim=2)
l = torch.cat(l_BLOCKS, dim=2)
m = torch.cat(m_BLOCKS, dim=2)

-----------Attn : Q0xK0---------

tensor([[[[ 0.7635, -0.7526,  0.0517, -0.1423],
          [ 1.1500,  1.3530, -0.5210, -0.5599],
          [ 0.8521, -0.1451,  0.1048, -0.1800]]]], grad_fn=<AddBackward0>)

tensor([[[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]]], grad_fn=<SplitBackward0>)

-----------Attn : Q1xK0---------

tensor([[[[ 0.7635, -0.7526,  0.0517, -0.1423],
          [ 1.1500,  1.3530, -0.5210, -0.5599],
          [ 0.8521, -0.1451,  0.1048, -0.1800]]]], grad_fn=<AddBackward0>)

...

tensor([[[[ 0.8079, -0.7476,  0.0581, -0.1573],
          [ 1.8556, -0.3380, -0.3364,  0.2566],
          [ 1.0651, -0.2864,  0.1042, -0.1167]]]], grad_fn=<AddBackward0>)

tensor([[[[ 1.1792,  1.5286, -0.5398, -0.5838],
          [ 0.8137, -0.3840,  0.1177, -0.1524],
          [ 0.8614, -0.0307,  0.1853, -0.1604]]]], grad_fn=<AddBackward0>)

-----------Attn : Q1xK1---------

tensor([[[[ 0.8079, -0.7476,  0.0581, -0.1573],
          [ 1.8556, -0.3380, -0.3364,  0.2566],
          [ 1.0651, -0.2864,  0.1042, -0.1167]]]], grad_fn=<AddBackward0>)

tensor([[[[ 1.2649,  1.0774, -0.3792, -0.4560],
          [ 1.3142, -0.5549,  0.0909, -0.2419],
          [ 1.2212, -0.3228,  0.1693, -0.2085]]]], grad_fn=<AddBackward0>)

# Flash Attention2

Flash Attntion2 相较1最终要的特点就是改变内外循环，从而减少O的交换

![](./flash2.png)

In [6]:
# author: 小冬瓜AIGC
import torch
from einops import rearrange

Q_BLOCK_SIZE = 3
KV_BLOCK_SIZE = 3
NEG_INF = -1e10  # -infinity
EPSILON = 1e-10
Q_LEN = 6
K_LEN = 6
Tr = Q_LEN // Q_BLOCK_SIZE # Q循环
Tc = K_LEN // KV_BLOCK_SIZE # KV 循环 

Q = torch.randn(1, 1, 6, 4, requires_grad=True).to(device='cpu')
K = torch.randn(1, 1, 6, 4, requires_grad=True).to(device='cpu')
V = torch.randn(1, 1, 6, 4, requires_grad=True).to(device='cpu')
O = torch.zeros_like(Q, requires_grad=True)
l = torch.zeros(Q.shape[:-1])[..., None]
m = torch.ones(Q.shape[:-1])[..., None] * NEG_INF

Q_BLOCKS = torch.split(Q, Q_BLOCK_SIZE, dim=2)
K_BLOCKS = torch.split(K, KV_BLOCK_SIZE, dim=2)
V_BLOCKS = torch.split(V, KV_BLOCK_SIZE, dim=2)
O_BLOCKS = list(torch.split(O, Q_BLOCK_SIZE, dim=2))
l_BLOCKS = list(torch.split(l, Q_BLOCK_SIZE, dim=2))
m_BLOCKS = list(torch.split(m, Q_BLOCK_SIZE, dim=2))

Flash Attention 每个步骤都要做scaled处理， Flash Attention2 只在最后做scaled

$$
\begin{align} O^{(2)}&=diag(l^{(1)}/l^{(2)})^{-1}O^{(1)}+diag(l^{(2)})^{-1}e^{S^{(2)}-m^{(2)}}V^{(2)} \end{align}
$$

![](./flash_scaled.png)_

$$
\begin{align} \widetilde{O}^{(2)} &= diag(l^{(1)})^{-1}O^{(1)}+e^{S^{(2)}-m^{(2)}}V^{(2)} \\ O^{(2)} &= diag(l^{(2)})^{-1}\widetilde{O}^{(2)} \\  O^{(N)} &= diag(l^{(N)})^{-1}\widetilde{O}^{(N)} \end{align}
$$

In [2]:
# start with Q
for i in range(Tr):
    Qi = Q_BLOCKS[i]
    Oi = O_BLOCKS[i]
    li = l_BLOCKS[i]
    mi = m_BLOCKS[i]
    
    for j in range(Tc):
        #if j>i: 
        #    continue    # ignore masked      
        Kj = K_BLOCKS[j]
        Vj = V_BLOCKS[j]

        S_ij = Qi @ Kj.transpose(2,3)
        m_block_ij, _ = torch.max(S_ij, dim=-1, keepdims=True)
        mi_new = torch.maximum(m_block_ij, mi)
        P_ij_hat = torch.exp(S_ij - mi_new)
        l_block_ij = torch.sum(P_ij_hat, dim=-1, keepdims=True) + EPSILON
        li_new = torch.exp(mi - mi_new) * li  + l_block_ij 
        O_i = torch.exp(mi - mi_new) * Oi + P_ij_hat @ Vj
          
        print(f'-----------O{i} = attn( Q{i}, KV[{j}])---------')
        print(O_i)
        
    O_BLOCKS[i] = O_i / li_new # 最后做Scaled
    l_BLOCKS[i] = li_new
    m_BLOCKS[i] = mi_new
    
O = torch.cat(O_BLOCKS, dim=2)
l = torch.cat(l_BLOCKS, dim=2)
m = torch.cat(m_BLOCKS, dim=2)

-----------O0 = attn( Q0, KV[0])---------

tensor([[[[-0.2135, -0.4783,  0.6653, -1.6356],
          [ 0.6038,  1.9621, -1.6474, -2.2015],
          [ 0.6543,  1.9929, -1.6862, -2.1673]]]], grad_fn=<AddBackward0>)

-----------O0 = attn( Q0, KV[1])---------

tensor([[[[ 0.7904, -0.9252, -2.3112, -1.1009],
          [ 0.7641, -0.9668, -2.3473, -1.1087],
          [-1.3296, -0.5212, -1.1518,  0.2756]]]], grad_fn=<AddBackward0>)

-----------O1 = attn( Q1, KV[0])---------

tensor([[[[ 0.3425,  1.8362, -1.4927, -2.2582],
          [ 1.3855,  1.3566, -1.3693, -0.5180],
          [-0.2295, -0.5334,  0.7115, -1.5742]]]], grad_fn=<AddBackward0>)

-----------O1 = attn( Q1, KV[1])---------

tensor([[[[-0.8458, -1.3041, -1.1890, -0.2464],
          [-0.8405, -1.0774, -1.6372, -0.2429],
          [ 1.0866, -0.8237, -2.0620, -1.1687]]]], grad_fn=<AddBackward0>)